# Training Small scale ChatGPT from scratch 

### dependencies:
- wget
- PyTorch

In [17]:
# imports
import wget
import torch

C:\Users\Angad Sandhu\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# getting the shakesper dataset
URL = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
res = wget.download(URL, "tinyshakespeare.txt")

In [4]:
# reading dataset to store in a variable
with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print("dataset size (in characters) : ", len(text))

dataset size (in characters) :  1115394


In [7]:
# looking at the sample o this data
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [11]:
# unique characters that occur in this text
all_chars = sorted(list(set(text)))
vocab_size = len(all_chars)

print(f"Vocabulary Size : {vocab_size}, with the charaters : {''.join(all_chars)}")

Vocabulary Size : 65, with the charaters : 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


## Pre-Processing

In [12]:
# creating mapping between characters to integers
stoi = { ch:i for i,ch in enumerate(all_chars) }
itos = { i:ch for i,ch in enumerate(all_chars) }

In [13]:
# function to transform encodings from one another 
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [16]:
# testing the encodings
print(f"encoding 'hello' into an integer array : {encode('hello')}")
print(f"decoding the array to get back 'hello' : {decode(encode('hello'))}")

encoding 'hello' into an integer array : [46, 43, 50, 50, 53]
decoding the array to get back 'hello' : hello


In [18]:
# storing the entire dataset into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:100]) # looking at datasample again

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


## Spliting Dataset into Test / Train / Validation 

In [ ]:
# spliting up the data into train and validation sets
n = int(0.9*len(data)) # 90% train, 10% val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

In [ ]:
print(xb) # our input to the transformer

## Training Model